In [79]:
import pandas as pd
import numpy as np
import random
from lorem_text import lorem

In [80]:
articles = pd.read_csv('article.csv', sep=';')
proceeding = pd.read_csv('proceeding_papers.csv', sep=';')

In [81]:
def generate_reviews(x):
    rev = x.split('|')
    reviews = ''
    for r in rev:
        if len(reviews) > 0:
            reviews += '|'
        reviews += lorem.words(10)
    return reviews

def generate_decisions(x):
    rev = x.split('|')
    decisions = ''
    for r in rev:
        if len(decisions) > 0:
            decisions += '|'
        decisions += 'Accepted'
    return decisions

def remove_space(x):
    if type(x) == float:
        return 'TheRiddler'
    else:
        return x.replace(' ','')

def generate_decisionIDs(art, rev):
    reviewers = rev.split('|')
    IDs = ''
    for i, r in enumerate(reviewers):
        if len(IDs) > 0:
            IDs += '|'
        IDs += f"review{art}_{i}" 
    return IDs

In [82]:
proceeding = proceeding.rename(columns = {'booktitle': 'conference',
                                          'inproctitle': 'articleTitle', 
                                          'proctitle': 'conferenceTitle',
                                          'keywords': 'topic',
                                          'inprocee': 'doi',
                                          'proceedings': 'conferenceID',
                                          'inproceedings': 'articleID'
                                         })



proceeding['conferenceID'] = proceeding.groupby(by=['conference', 'procmdate', 'conferenceTitle']).ngroup().add(1)
confID = proceeding['conferenceID'].unique()
authors = proceeding['author'].unique()

a = set([])
for aut in authors:
    if type(aut) == float:
        continue
    for au in aut.split('|'):
        a.add(au)

names = set([])
for author in a:
    for name in author.split(' '):
        names.add(name)

confNamePair = {}
confTypePair = {}
confType = ['workshop', 'symposium', 'expert_group', 'regular']
for ID in confID: 
    confNamePair[ID] = f'{random.sample(names, k=1)[0]} {random.sample(names, k=1)[0]}'
    confTypePair[ID] = random.sample(confType, k=1)[0]
    
def get_author(x):
    return confNamePair[x]

def get_type(x):
    return confTypePair[x]

proceeding['chair'] = proceeding['conferenceID'].apply(get_author)
proceeding['conferenceType'] = proceeding['conferenceID'].apply(get_type)

pro = set(proceeding['articleID'].unique())
confPaperTypePair = {}
confPaperTypes = ['poster', 'full_paper', 'demo_paper', 'short_paper']
for artID in pro:
    confPaperTypePair[artID] = random.sample(confPaperTypes, k=1)[0]
    
def get_paper_type(x):
    return confPaperTypePair[x]

proceeding['articleType'] = proceeding['articleID'].apply(get_paper_type)

proceeding['reviews'] = proceeding['reviewed_by'].apply(generate_reviews)
proceeding['decisions'] = proceeding['reviewed_by'].apply(generate_decisions)
proceeding['decisionID'] = proceeding.apply(lambda x: generate_decisionIDs(x.articleID, x.reviewed_by), axis=1)

proceeding['author'] = proceeding['author'].apply(remove_space)
proceeding['conference'] = proceeding['conference'].apply(remove_space)
proceeding['reviewed_by'] = proceeding['reviewed_by'].apply(remove_space)
proceeding['chair'] = proceeding['chair'].apply(remove_space)


proceeding = proceeding.drop(['volume',
                                 'url', 
                                 'author-orcid',
                                 'ee-type', 
                                 'pages', 
                                 'inprocmdate', 
                                 'author-orcid', 
                                 'abstract', 
                                 'co_authors',
                                 'citations',
                                 'isbn',
                                 'series',
                                 'publisher',
                                 'doi',
                                 'crossref',
                                 'key',
                                 'procee',
                                 'corresponding', 
                                 'procmdate'], 1)

proceeding.to_csv('proceeding_processed.csv')
proceeding[:5].to_csv('proceeding_slice.csv')

In [83]:
articles = articles.rename(columns = {'keywords': 'topics'})

articles['journalID'] = articles.groupby(by=['journal', 'volume']).ngroup().add(1)
jourID = articles['journalID'].unique()
authors = articles['author'].unique()

a = set([])
for aut in authors:
    if type(aut) == float:
        continue
    for au in aut.split('|'):
        a.add(au)

names = set([])
for author in a:
    for name in author.split(' '):
        names.add(name)
        
jourNamePair = {}
for ID in jourID:
    jourNamePair[ID] = f'{random.sample(names, k=1)[0]} {random.sample(names, k=1)[0]}'

def get_author(x):
    return jourNamePair[x]

articles['editor'] = articles['journalID'].apply(get_author)    


art = set(articles['article'].unique())
    
jourPaperTypePair = {}

jourPaperTypes = ['full_paper', 'demo_paper', 'short_paper']
for artID in art:
    jourPaperTypePair[artID] = random.sample(jourPaperTypes, k=1)[0]

def get_paper_type(x):
    return jourPaperTypePair[x]    

articles['paperType'] = articles['article'].apply(get_paper_type)

articles['reviews'] = articles['reviewed_by'].apply(generate_reviews)
articles['decisions'] = articles['reviewed_by'].apply(generate_decisions)
articles['decisionID'] = articles.apply(lambda x: generate_decisionIDs(x.article, x.reviewed_by), axis=1)

articles['author'] = articles['author'].apply(remove_space)
articles['journal'] = articles['journal'].apply(remove_space)
articles['reviewed_by'] = articles['reviewed_by'].apply(remove_space)
articles['editor'] = articles['editor'].apply(remove_space)


articles = articles.drop(['key',
                          'author-orcid',
                          'ee',
                          'ee-type',
                          'pages',
                          'publtype',
                          'abstract',
                          'citations',
                          'url',
                          'key',
                          'corresponding',
                          'co_authors',
                          'correspondingID',
                          'mdate'],1)

articles.to_csv('articles_processed.csv')
articles[:5].to_csv('article_slice.csv')

In [91]:
proceeding.head()

,articleID,author,conference,articleTitle,year,reviewed_by,conferenceID,conferenceTitle,location,topic,chair,conferenceType,articleType,reviews,decisions,decisionID
0,7043780,JoxeGaintzarain|MarisaNavarro|MontserratHermo,MFCS,Goals in the Propositional Horn Language Are M...,2005,HarishChander|HuiZhang0001|MinLiu0008,1283,Mathematical Foundations of Computer Science 1...,Oosterbeek,pariatur|ea|data modeling|data querying,ClimerNinghui,workshop,demo_paper,cum omnis aliquid ad quibusdam dolorum assumen...,Accepted|Accepted|Accepted,review7043780_0|review7043780_1|review7043780_2
1,7043780,JoxeGaintzarain|MarisaNavarro|MontserratHermo,MFCS,Goals in the Propositional Horn Language Are M...,2005,HarishChander|HuiZhang0001|MinLiu0008,1286,45th International Symposium on Mathematical F...,Kviv,pariatur|ea|data modeling|data querying,HengfeiVatche,symposium,demo_paper,earum dolorem aliquid ad accusamus nulla bland...,Accepted|Accepted|Accepted,review7043780_0|review7043780_1|review7043780_2
2,7043780,JoxeGaintzarain|MarisaNavarro|MontserratHermo,MFCS,Goals in the Propositional Horn Language Are M...,2005,HarishChander|HuiZhang0001|MinLiu0008,1285,43rd International Symposium on Mathematical F...,Marioepol,pariatur|ea|data modeling|data querying,HazlettKugler,symposium,demo_paper,amet in mollitia illo laboriosam cupiditate is...,Accepted|Accepted|Accepted,review7043780_0|review7043780_1|review7043780_2
3,7043780,JoxeGaintzarain|MarisaNavarro|MontserratHermo,MFCS,Goals in the Propositional Horn Language Are M...,2005,HarishChander|HuiZhang0001|MinLiu0008,1284,Mathematical Foundations of Computer Science 1...,Oosterbeek,pariatur|ea|data modeling|data querying,McElweeHauptmann,workshop,demo_paper,quia explicabo quis accusantium nam veniam exc...,Accepted|Accepted|Accepted,review7043780_0|review7043780_1|review7043780_2
4,7042963,DaniëlPaulusma|GiacomoPaesani|KonradK.Dabrowsk...,MFCS,"On the Price of Independence for Vertex Cover,...",2018,KiaDashtipour|AlexanderV.Terekhov|N.Kelly-Boxall,1283,Mathematical Foundations of Computer Science 1...,Oosterbeek,praesentium|at|data management|data storage,ClimerNinghui,workshop,full_paper,nesciunt nam id repellendus ab provident maxim...,Accepted|Accepted|Accepted,review7042963_0|review7042963_1|review7042963_2
